# Import necessary depencencies
While most of them are mentioned in each section, you need to make sure you have Pandas, NumPy, SciPy, and Scikit-Learn installed, which are used for data processing and machine learning.

Deep learning frameworks used in this chapter include Keras with the TensorFlow backend.

NLP libraries we use include spaCy, NLTK, and Gensim. We also use our custom developed text preprocessing and normalization module, which you can find in the files named contractions.py and text_normalizer.py.

Utilities related to supervised model fitting, prediction, and evaluation are present in model_evaluation_utils.py, so make sure you place these modules in the same directory and the other Python files and Jupyter notebooks for this chapter.

In [2]:
# Python libraries and frameworks specific to text analytics, NLP, and machine learning
import pandas as pd
import numpy as np
import text_normalizer as tn
# import text_normalizer_el as tn
import model_evaluation_utils as meu
import nltk
import textblob

np.set_printoptions(precision=2, linewidth=80)

# Load and normalize data

In [6]:
# 15,000 reviews
dataset = pd.read_csv('movie_reviews1.csv') # read_csv(...) utility function from pandas

reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

# extract data for model evaluation
test_reviews = reviews[3:]
test_sentiments = sentiments[3:]
sample_review_ids = [1, 2, 3]

ParserError: Error tokenizing data. C error: Expected 2 fields in line 3, saw 3


In [13]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', textblob.TextBlob(review).sentiment.polarity)
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
Predicted Sentiment polarity: -0.3625
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
Predicted Sentiment polarity: 0.16666666666666674
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot
Actual Sentiment: positive
Predicted Sentiment polarity: -0.037239583333333326
------------------------------------------------------------


In [14]:
sentiment_polarity = [textblob.TextBlob(review).sentiment.polarity for review in test_reviews]

In [15]:
# threshold of 0.1 based on multiple experiments.
predicted_sentiments = ['positive' if score >= 0.1 else 'negative' for score in sentiment_polarity]

In [16]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])

# We get an overall F1-score and accuracy of 77%, which is good considering it’s an unsupervised model

Model Performance metrics:
------------------------------
Accuracy: 0.7669
Precision: 0.767
Recall: 0.7669
F1 Score: 0.7668

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.76      0.78      0.77      7510
    negative       0.77      0.76      0.76      7490

    accuracy                           0.77     15000
   macro avg       0.77      0.77      0.77     15000
weighted avg       0.77      0.77      0.77     15000


Prediction Confusion Matrix:
------------------------------


TypeError: __new__() got an unexpected keyword argument 'labels'

# Sentiment Analysis with AFINN
AFINN lexicon is perhaps one of the simplest and most popular lexicons and can be used extensively for sentiment analysis. The current version of the lexicon is AFINN-en-165.txt and it contains over 3,300 words with a polarity score associated with each word.

A library on top of this in Python called afinn. You can import the library and instantiate an object using the following code.

In [ ]:
!pip install afinn
from afinn import Afinn

afn = Afinn(emoticons=True) 

## Predict sentiment for sample reviews
We can now use this object and compute the polarity of our chosen four sample
reviews using the following snippet.

In [ ]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-'*60)

We can compare the actual sentiment label for each review and check out the
predicted sentiment polarity score. A negative polarity typically denotes negative sentiment. I used a threshold of >= 1.0 to determine if the overall sentiment is positive. You can choose your own threshold based on analyzing your own corpora.

## Predict sentiment for test dataset

In [ ]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = ['positive' if score >= 1.0 else 'negative' for score in sentiment_polarity]

## Evaluate model performance
Now that we have our predicted sentiment labels, we can evaluate our model
performance based on standard performance metrics using our utility function. See

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])

# Sentiment Analysis with SentiWordNet
The WordNet corpus is one of the most popular corpora for the English language and is used extensively in natural language processing and semantic analysis. WordNet gave us the concept of synsets or synonym sets. SentiWordNet lexicon is based on WordNet synsets and can be used for sentiment analysis and opinion mining.

In [28]:
# We use the NLTK library, which provides a Pythonic interface into SentiWordNet
from nltk.corpus import sentiwordnet as swn

awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


## Build model
Let’s now build a generic function to extract and aggregate sentiment scores for a complete textual document based on matched synsets in that document.

In [31]:
def analyze_sentiment_sentiwordnet_lexicon(review,
                                           verbose=False):

    # tokenize and POS tag text tokens
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tags
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')):
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')):
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')):
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')):
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        # if senti-synset is found        
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                             ['Predicted Sentiment', 'Objectivity',
                                                              'Positive', 'Negative', 'Overall']], 
                                                             labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
        
    return final_sentiment

Our function takes in a movie review, tags each word with its corresponding POS tag, extracts the sentiment scores for any matched synset token based on its POS tag, and finally aggregates the scores. This process will be clearer when we run it on our sample documents.

## Predict sentiment for sample reviews

In [32]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_sentiwordnet_lexicon(review, verbose=True)    
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative


TypeError: __new__() got an unexpected keyword argument 'labels'

We can clearly see the predicted sentiment along with sentiment polarity scores and
an objectivity score for each sample movie review depicted in formatted dataframes.
Let’s use this model to predict the sentiment of all our test reviews and evaluate its
performance. A threshold of >=0 has been used for the overall sentiment polarity to be
classified as positive (whereas < 0 is a negative sentiment.

## Predict sentiment for test dataset

In [33]:
norm_test_reviews = tn.normalize_corpus(test_reviews)
predicted_sentiments = [analyze_sentiment_sentiwordnet_lexicon(review, verbose=False) for review in norm_test_reviews]

## Evaluate model performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])

We get an overall F1-score of 60%, which is definitely a step down from our previous models. Maybe playing around with the thresholds here might help.

# Sentiment Analysis with VADER
Based on a rule-based sentiment analysis framework, specifically tuned to analyze sentiments in social media.

In [19]:
# You can use the library based on NLTK’s interface under the nltk.sentiment.vader module
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Build model

In [20]:
def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):
    # pre-process text
    review = tn.strip_html_tags(review)
    review = tn.remove_accented_chars(review)
    review = tn.expand_contractions(review)
    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

## Predict sentiment for sample reviews

In [21]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!
Actual Sentiment: negative
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            negative           -0.8     0.0%    40.0%   60.0%
------------------------------------------------------------
REVIEW: I don't care if some people voted this movie to be bad. If you want the Truth this is a Very Good Movie! It has every thing a movie should have. You really should Get this one.
Actual Sentiment: positive
     SENTIMENT STATS:                                                     
  Predicted Sentiment Polarity Score Positive             Negative Neutral
0            negative          -0.16    16.0%  14.000000000000002%   69.0%
------------------------------------------------------------
REVIEW: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unb

## Predict sentiment for test dataset

In [28]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) for review in test_reviews]

## Evaluate model performance

In [29]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.711
Precision: 0.7236
Recall: 0.711
F1 Score: 0.7068

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.67      0.83      0.74      7510
    negative       0.78      0.59      0.67      7490

   micro avg       0.71      0.71      0.71     15000
   macro avg       0.72      0.71      0.71     15000
weighted avg       0.72      0.71      0.71     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6235     1275
        negative       3060     4430


DARAS...


In [23]:
# coding: utf8
import spacy
from spacy import displacy
import pandas as pd
from collections import defaultdict
import operator
indexes = {}
df = pd.read_csv('greek_sentiment_lexicon.tsv',sep='\t', encoding='utf-8-sig')
df = df.fillna('N/A')
for index, row in df.iterrows():
    df.at[index, "Term"] = row["Term"].split(' ')[0]
    indexes[df.at[index, "Term"]] = index


text = '''Έχω μείνει έκπληκτος! Πώς γίνεται αυτό; Η έκπληξη είναι τόσο μεγάλη! Α, τώρα εξηγούνται όλα.'''
subj_scores = {
    'OBJ': 0,
    'SUBJ-': 0.5,
    'SUBJ+': 1,
}

emotion_scores = {
    'N/A': 0,
    '1.0': 0.2,
    '2.0': 0.4,
    '3.0': 0.6,
    '4.0': 0.8,
    '5.0': 1,
}

polarity_scores = {
    'N/A': 0,
    'BOTH': 0,
    'NEG': -1,
    'POS': 1
}

# nlp = spacy.load('el')
import el_core_news_md
nlp = el_core_news_md.load()
doc = nlp(text)
subjectivity_score = 0
anger_score = 0
disgust_score = 0
fear_score =  0
happiness_score = 0
sadness_score = 0
surprise_score = 0
matched_tokens = 0
for token in doc:
    lemmatized_token = token.lemma_
    if (lemmatized_token in indexes):
        indx = indexes[lemmatized_token]
        pos_flag = False
        for col in ["POS1", "POS2", "POS3", "POS4"]:
            if (token.pos_ == df.at[indx,col]):
                pos_flag = True
                break
        if (pos_flag == True):
            match_col_index = [int(s) for s in col if s.isdigit()][0]
            subjectivity_score += subj_scores[df.at[indx,'Subjectivity'+str(match_col_index)]]
            anger_score += emotion_scores[str(df.at[indx, 'Anger'+str(match_col_index)])]
            disgust_score += emotion_scores[str(df.at[indx, 'Disgust'+str(match_col_index)])]
            fear_score += emotion_scores[str(df.at[indx, 'Fear'+str(match_col_index)])]
            happiness_score += emotion_scores[str(df.at[indx, 'Happiness'+str(match_col_index)])]
            sadness_score += emotion_scores[str(df.at[indx,'Sadness'+str(match_col_index)])]
            surprise_score += emotion_scores[str(df.at[indx, 'Surprise'+str(match_col_index)])]
            matched_tokens+=1
            for child in token.children:
                print(child, child.dep_)

try:
    print('Subjectivity: ' + str(subjectivity_score/matched_tokens * 100)+'%')
    emotions = {'anger': anger_score, 'disgust': disgust_score, 'fear':fear_score, 'happiness':happiness_score, 'sadness': sadness_score, 'surprise': surprise_score}
    emotion = max(emotions.items(), key=operator.itemgetter(1))[0]
    if (emotions[emotion] == 0):
        print('Unable to detect emotion')
    else:
        print('Main emotion: ' + emotion + '. Emotion score: ' + str(emotions[emotion]*100/matched_tokens) + '%')
except:
    print('No matched tokens')

FileNotFoundError: [Errno 2] No such file or directory: 'greek_sentiment_lexicon.tsv'